In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt

import sys; sys.path.insert(0, '../') #adds all the code we've written in src
from preprocessing import *

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Ayush\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Ayush\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Ayush\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Ayush\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
df = pd.read_csv("../../cyberbullying_tweets/cyberbullying_tweets.csv")
df

,tweet_text,cyberbullying_type
0,"In other words #katandandre, your food was cra...",not_cyberbullying
1,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying
2,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying
3,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying
4,@RudhoeEnglish This is an ISIS account pretend...,not_cyberbullying
...,...,...
47687,"Black ppl aren't expected to do anything, depe...",ethnicity
47688,Turner did not withhold his disappointment. Tu...,ethnicity
47689,I swear to God. This dumb nigger bitch. I have...,ethnicity
47690,Yea fuck you RT @therealexel: IF YOURE A NIGGE...,ethnicity


In [11]:
%time df["processed_text"] = preprocess_all(df.tweet_text)

Wall time: 28.8 s


In [12]:
df

,tweet_text,cyberbullying_type,processed_text
0,"In other words #katandandre, your food was cra...",not_cyberbullying,"[word, katandandre, food, crapilicious]"
1,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying,"[aussietv, white, theblock, imacelebrityau, to..."
2,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying,"[xochitlsuckkks, classy, whore, red, velvet, c..."
3,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying,"[jason_gio, meh, p, thanks, head, concerned, a..."
4,@RudhoeEnglish This is an ISIS account pretend...,not_cyberbullying,"[rudhoeenglish, isi, account, pretending, kurd..."
...,...,...,...
47687,"Black ppl aren't expected to do anything, depe...",ethnicity,"[black, ppl, expected, anything, depended, any..."
47688,Turner did not withhold his disappointment. Tu...,ethnicity,"[turner, withhold, disappointment, turner, cal..."
47689,I swear to God. This dumb nigger bitch. I have...,ethnicity,"[swear, god, dumb, nigger, bitch, got, bleach,..."
47690,Yea fuck you RT @therealexel: IF YOURE A NIGGE...,ethnicity,"[yea, fuck, therealexel, nigger, fucking, unfo..."
